In [ ]:
from GlobalSets.Mongo import Clients as MongoClient, DataBases as db, Collections as col
print(MongoClient.RemoteUnitClient['CeDRI_UGV_dashboard']['config'].find_one_and_update(upsert=True, filter={'user': "default"}, update= { '$set': { "Branch" : 'ECE'} }, ))

In [8]:
# Fill Nodes default info
Nodes = [{
        'node'    : '/odom',
        'msg'     : 'Odometry',
    },
    {
        'node'    : '/oadom',
        'msg'     : 'Odoasmetry',
    }]

DATASOURCE = 'a'
DATALAKE = 'b'

def fillNode(nodes):
    # Deafult values
    def defaultValues(node):
        # Create the default values dictionary
        try:
            _return = {
                'sleep'    : 1,                     # Sleep of 1 second
                'callback': None,                   # No callback function
                'dataPath': {
                    'dataSource': DATASOURCE,       # Add DATASOURCE of config.py
                    'dataBase'  : DATALAKE,         # Add DATALAKE of config.py
                    'collection': node['node']      # Collection name is the same of of node name
                }
            }
            return _return
        except Exception as e:
            rospy.logerr("Error in default values")
            rospy.logerr("An exception occurred:", type(e).__name__,e.args)
            return None
    
    # Update the dictionary
    def update_nested_dict(default, target):
        # Without default values
        if default == None:
            rospy.logerr("No default dictionary for: ", target)
            target = None
        # Run for all dictionary
        for key, value in default.items():
            try:
                # Check if nested
                if isinstance(value, dict) and key in target and isinstance(target[key], dict):
                    update_nested_dict(value, target[key])
                else:
                    target.setdefault(key, value)
            except Exception as e:
                rospy.logerr("Error in the key:", key, value)
                rospy.logerr("An exception occurred:", type(e).__name__,e.args)

    # Copy Nodes
    try:
        _nodes = nodes.copy()
    except Exception as e:
        rospy.logerr("Error in copy of NODES")
        rospy.logerr("An exception occurred:", type(e).__name__,e.args)

    # Run for all codes        
    for node in _nodes:
        try:
            update_nested_dict(defaultValues(node=node), node)
        except Exception as e:
            rospy.logerr("Error in set values of:", node)
            rospy.logerr("An exception occurred:", type(e).__name__,e.args)

    return _nodes
               
print(fillNode(nodes=Nodes))

[{'node': '/odom', 'msg': 'Odometry', 'sleep': 1, 'callback': None, 'dataPath': {'dataSource': 'a', 'dataBase': 'b', 'collection': '/odom'}}, {'node': '/oadom', 'msg': 'Odoasmetry', 'sleep': 1, 'callback': None, 'dataPath': {'dataSource': 'a', 'dataBase': 'b', 'collection': '/oadom'}}]


In [5]:
#!/usr/bin/env python3
from GlobalSets.Mongo import Clients as MongoClient, DataBases as db, Collections as col
import math, bson, json, datetime, time, asyncio, threading
import ast
import threading

Scripts = MongoClient.RemoteUnitClient['CeDRI_dashboard']['scripts']
Logs = MongoClient.RemoteUnitClient['CeDRI_dashboard']['logs']
threads = []

def my_handler(x):
    if isinstance(x, datetime.datetime):
        return x.isoformat()
    elif isinstance(x, bson.objectid.ObjectId):
        return str(x)
    elif isinstance(x, float) and math.isnan(x):
        return None
    else:
        print(x)
        print('_______________________________________________________________________________________________________')
        raise TypeError(x)
def nan2None(obj):
    if isinstance(obj, datetime.datetime):
        return obj.isoformat()
    elif isinstance(obj, bson.objectid.ObjectId):
        return str(obj)
    elif isinstance(obj, dict):
        return {k:nan2None(v) for k,v in obj.items()}
    elif isinstance(obj, list):
        return [nan2None(v) for v in obj]
    elif isinstance(obj, float) and math.isnan(obj):
        return None
    return obj
class NanConverter(json.JSONEncoder):
    def default(self, obj):
        my_handler(obj)
        pass
    def encode(self, obj, *args, **kwargs):
        obj = nan2None(obj)
        return super().encode(obj, *args, **kwargs)
    def iterencode(self, obj, *args, **kwargs):
        obj = nan2None(obj)
        return super().iterencode(obj, *args, **kwargs)

def log(robot, script, msg, type):
    out = {
        'script': script,
        'robot': robot,
        'msg': msg,
        'type': type,
        'datetime': datetime.datetime.now()
    }
    Logs.insert_one(document=out).acknowledged

def rospub(robot, topic, comand):
    return True

def action(robot, command):
    return True

def updateCode(metaCode, set):
    return Scripts.update_one(filter={'name': metaCode['name']}, update={'$set': set}).acknowledged

def nextExec(metaCode):
    _now = datetime.datetime.now()
    set = {
        'next':_now + datetime.timedelta(seconds=metaCode['sample']), 
        'last': _now
    }
    updateCode(metaCode=metaCode, set=set)

def statusExec(metaCode, status):
    set = {
        'status': status
    }
    return updateCode(metaCode=metaCode, set=set)

def runCode(metaCode):
    try:
        _id = metaCode['name']
        code = metaCode['code']
        robot = metaCode['robot']     
        code = code.replace('log(', 'log(robot="' + robot + '",script="' + _id + '",' )
    except Exception as e:
        log(robot=robot, msg=e,type='error')
        statusExec(metaCode=metaCode, status='error')

    try:
        code = compile(code, "<string>", "exec")
    except Exception as e:
        log(robot=robot, msg=e,type='error')
        statusExec(metaCode=metaCode, status='error')

    try:
        statusExec(metaCode=metaCode, status='run')
        exec(code)
        if Scripts.find_one(filter={'name': metaCode['name']})['status'] == 'run':
            statusExec(metaCode=metaCode, status='wait')
    except Exception as e:
        log(robot=robot, msg=e,type='error')
        statusExec(metaCode=metaCode, status='error')

def nextSleep(max = 10):
    pipeline = [
        {
            '$match': {
                'status': {
                    '$ne': 'error'
                }
            }
        }, {
            '$group': {
                '_id': None, 
                'next': {
                    '$min': '$next'
                }
            }
        }
    ]
    nextExe = list(Scripts.aggregate(pipeline=pipeline))[0]['next'].timestamp()
    now = datetime.datetime.now().timestamp()
    wait = (nextExe - now)*(nextExe > now)
    if wait > 10:
        wait = 10
    if wait < 1: 
        wait = 1
    return wait

def runCodeAsync(metaCode):
    thread = threading.Thread(target=runCode, args=(metaCode,))
    thread.setName(metaCode['name'])
    thread.daemon = True
    thread.start()
    _thread = {
        'thread': thread,
        'name': metaCode['name']
    }
    threads.append(_thread)

def cleanThreads():
    for thread in threads:
        if not thread['thread'].is_alive():
            threads.remove(thread)
    return True

def stopThread(metaCode):
    _threads = [thread for thread in threads if thread.get('name') == metaCode['name']]
    for thread in _threads:
        thread['thread'].join()

while True:
    cleanThreads()
    codes = list(Scripts.find(filter={}))
    for code in codes:
        try:
            if(code['status'] == 'stop' or code['status'] == 'error'):
                stopThread(metaCode=code)
            if datetime.datetime.now() > code['next'] and code['status'] == 'wait':
                nextExec(metaCode=code)
                runCodeAsync(metaCode=code)
        except Exception as e:
            log(robot=code['name'], msg=e,type='error')
            statusExec(metaCode=code, status='error')
    time.sleep(nextSleep())

        


KeyboardInterrupt: 

In [76]:
#!/usr/bin/env python3
from GlobalSets.Mongo import Clients as MongoClient, DataBases as db, Collections as col
import math, bson, json, datetime, time, asyncio, threading
import ast
import threading

Client = MongoClient.RemoteUnitClient
Scripts = Client['CeDRI_dashboard']['scripts']
Logs = Client['CeDRI_dashboard']['logs']
threads = []

robots = list(Client['CeDRI_dashboard']['robots'].find())
pipeline = [
    {
        '$unwind': {
            'path': '$list', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$match': {
            'list.ip_target': '192.168.217.183'
        }
    }, {
        '$sort': {
            'dateTime': -1
        }
    }, {
        '$limit': 1
    }
]
for robot in robots:
    try:
        connection = list(Client[robot['database']]['/connectionStatus'].aggregate(pipeline=pipeline))[0]
        update = [
            {
                '$set': {
                    'status.rtt_avg': connection['list']['rtt_avg'], 
                    'status.is_alive':  (datetime.datetime.now() - connection['dateTime']) < datetime.timedelta(seconds=15) and connection['list']['is_alive'],
                    'status.lastCheck': connection['dateTime']
                }
            }
        ]
        Client['CeDRI_dashboard']['robots'].update_one(
            filter={'name': robot['name']}, 
            update=update
        ).acknowledged
    except:
            pass



In [ ]:
#!/usr/bin/env python3
from GlobalSets.Mongo import Clients as MongoClient, DataBases as db, Collections as col
import math, bson, json, datetime, time, asyncio, threading
import ast
import threading

Client = MongoClient.RemoteUnitClient
Scripts = Client['CeDRI_dashboard']['scripts']
Logs = Client['CeDRI_dashboard']['logs']
threads = []
robot = '6450d1209e158f8fb6e3ac11'


_robot = list(Client['CeDRI_dashboard']['robots'].find_one(filter={'name': robot}))

pipeline = [
    {
        '$unwind': {
            'path': '$list', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$match': {
            'list.ip_target': '192.168.217.183'
        }
    }, {
        '$sort': {
            'dateTime': -1
        }
    }, {
        '$limit': 1
    }
]

connection = list(Client[_robot['database']]['/connectionStatus'].aggregate(pipeline=pipeline))[0]

update = [
    {
        '$set': {
            'status.rtt_avg': connection['list']['rtt_avg'], 
            'status.is_alive':  (datetime.datetime.now() - connection['dateTime']) < datetime.timedelta(seconds=15) and connection['list']['is_alive'],
            'status.lastCheck': connection['dateTime']
        }
    }
]

Client['CeDRI_dashboard']['robots'].update_one(
    filter={'name': _robot['name']}, 
    update=update
).acknowledged


